## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Santa Rosa,PH,14.3122,121.1114,80.01,83,20,3.44,few clouds
1,1,Mataura,NZ,-46.1927,168.8643,52.43,88,99,1.88,overcast clouds
2,2,Codajas,BR,-3.8367,-62.0569,89.13,60,72,4.65,moderate rain
3,3,Melo,UY,-32.3667,-54.1833,84.94,46,0,2.10,clear sky
4,4,San Patricio,US,28.0170,-97.5169,64.99,59,1,13.80,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 40
What is the maximum temperature you would like for your trip? 75


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Mataura,NZ,-46.1927,168.8643,52.43,88,99,1.88,overcast clouds
4,4,San Patricio,US,28.0170,-97.5169,64.99,59,1,13.80,clear sky
7,7,Bredasdorp,ZA,-34.5322,20.0403,69.80,52,89,10.36,overcast clouds
9,9,Matata,NZ,-37.8833,176.7500,59.00,91,76,1.01,broken clouds
10,10,Mar Del Plata,AR,-38.0023,-57.5575,64.40,82,90,8.05,moderate rain
13,13,Moree,AU,-29.4667,149.8500,66.20,82,0,6.91,clear sky
17,17,Ponta Do Sol,PT,32.6667,-17.1000,61.00,69,85,7.00,overcast clouds
18,18,Jieshi,CN,22.8134,115.8257,63.01,81,0,11.63,clear sky
21,21,Busselton,AU,-33.6500,115.3333,63.00,98,100,14.67,overcast clouds
22,22,Bluff,NZ,-46.6000,168.3333,54.34,89,100,4.88,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                221
City                   221
Country                219
Lat                    221
Lng                    221
Max Temp               221
Humidity               221
Cloudiness             221
Wind Speed             221
Current Description    221
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna().count()

City_ID                219
City                   219
Country                219
Lat                    219
Lng                    219
Max Temp               219
Humidity               219
Cloudiness             219
Wind Speed             219
Current Description    219
dtype: int64

In [12]:
preferred_cities_clean_df = preferred_cities_df.dropna()

In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Mataura,NZ,52.43,overcast clouds,-46.1927,168.8643,
4,San Patricio,US,64.99,clear sky,28.0170,-97.5169,
7,Bredasdorp,ZA,69.80,overcast clouds,-34.5322,20.0403,
9,Matata,NZ,59.00,broken clouds,-37.8833,176.7500,
10,Mar Del Plata,AR,64.40,moderate rain,-38.0023,-57.5575,
13,Moree,AU,66.20,clear sky,-29.4667,149.8500,
17,Ponta Do Sol,PT,61.00,overcast clouds,32.6667,-17.1000,
18,Jieshi,CN,63.01,clear sky,22.8134,115.8257,
21,Busselton,AU,63.00,overcast clouds,-33.6500,115.3333,
22,Bluff,NZ,54.34,overcast clouds,-46.6000,168.3333,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
  
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [16]:
# 7. Drop the rows where there is no Hotel Name.  There are no missing hotel names.
hotel_df.count()


City                   219
Country                219
Max Temp               219
Current Description    219
Lat                    219
Lng                    219
Hotel Name             219
dtype: int64

In [18]:
# 8a. Create the output File (CSV)
# Create the output file (CSV).
output_data_file = "WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.

info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))